In [1]:
username = "chat.Rotciv"
password = "%PKYHjmYYvd4b%&mGCqt"

In [2]:
print(repr(password))


'%PKYHjmYYvd4b%&mGCqt'


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

import undetected_chromedriver as uc


In [4]:
# Function 1: Initialize WebDriver and open Duolingo
def open_duolingo():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")  # Optional hardening
    driver = uc.Chrome(options=options)

    driver.get("https://www.duolingo.com")

    try:
        cookies_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-reject-all-handler'))
        )
        cookies_button.click()
    except:
        pass  # In case the cookies button doesn't show

    return driver

In [5]:
# Function 2: Log in to Duolingo
def login_duolingo(driver, username, password):
    try:
        # Wait for the "I already have an account" button and click it
        have_account_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[1]/header/div[2]/div[2]/div/button'))
        )
        have_account_button.click()

        # Fill in the username and password fields
        username_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="web-ui1"]'))
        )
        username_input.clear()  # Clear existing text
        username_input.send_keys(username)

        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="web-ui2"]'))
        )
        # pw_input.clear()
        time.sleep(1)
        
        pw_input.send_keys(password)

        # Click the login button
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="overlays"]/div[3]/div/div/form/div[1]/button'))
        )
        login_button.click()

        return driver
    except Exception as e:
        print(f"Error during login: {e}")
        return None

In [6]:
# Function 3: Navigate to Practice Hub
def navigate_to_practice_hub(driver):
    try:
        # Handle data consent popup
        data_consent_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/div[2]/div[2]/div[2]/button[1]'))
        )
        data_consent_button.click()
    except:
        pass
    try:
        # Click on the practice hub button
        practice_hub_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[2]/div/div[1]/div[2]/div[2]/a/span/div/img'))
        )
        practice_hub_button.click()
    except:
        pass

    try:
        # Click the "Words" button
        words_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[2]/div/div[3]/div/div[2]/div/div[2]/div/div[2]/button[1]'))
        )
        words_button.click()
    except:
        pass
    
    try:
        return driver
    except Exception as e:
        print(f"Error during navigation: {e}")
        return None

In [7]:
# Function 4: Load more words and extract
def extract_words(driver):
    try:
        # Wait for the element and get its text
        h1_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/div/div[3]/div/div[2]/div/section[1]/div/div/h1'))
        )
        language_text = h1_element.text
        match = re.search(r'Practice your (.+?) words', language_text)
        if match:
            learn_language = match.group(1)
        while True:
            try:
                # Find and click the 'Load more' button
                load_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//b[text()='Load more']/.."))
                )
                load_more_button.click()
            except TimeoutException:
                break

        # Get the page source after all words are loaded
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all h3 tags and their corresponding p tags
        h3_tags = soup.find_all('h3')
        word_pairs = []
        for h3 in h3_tags:
            p = h3.find_next_sibling('p')
            if p:
                word_pairs.append((h3.text.strip(), p.text.strip()))

        # Convert to a DataFrame
        df = pd.DataFrame(word_pairs, columns=[learn_language, 'Translation'])
        return df

    except Exception as e:
        print(f"Error during extraction: {e}")
        return None

In [8]:
if __name__ == "__main__":
    # Open Duolingo and get the driver
    driver = open_duolingo()

    # Pause and wait for manual login
    input("Please log in manually in the browser, then press ENTER here to continue...")

    # Navigate to the Practice Hub
    driver = navigate_to_practice_hub(driver)
    if not driver:
        print("Navigation to practice hub failed!")
        exit()

    # Extract words and save to DataFrame
    df = extract_words(driver)
    if df is not None:
        print(df.head())  # Display the first few rows of the DataFrame
    else:
        print("Extraction failed!")

     Spanish Translation
0  recetario    cookbook
1      pulpo     octopus
2      hongo    mushroom
3  incomible    inedible
4    meterse      broken


In [9]:
print(df)

df.to_json(r"C:\Users\victo\VSCode Folder\Duolingo\DuolingoSupport\Data\spanish.json")

        Spanish            Translation
0     recetario               cookbook
1         pulpo                octopus
2         hongo               mushroom
3     incomible               inedible
4       meterse                 broken
...         ...                    ...
4867         la         the, your, her
4868       niña  girl, child, daughter
4869         un             a, an, one
4870         el         the, your, her
4871      mujer      woman, wife, lady

[4872 rows x 2 columns]
